<a href="https://colab.research.google.com/github/orleansmartins/public/blob/main/Fatores_Nefin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de Pacotes

In [ ]:
!pip install --upgrade xlrd
!pip install investpy

In [ ]:
!pip install quandl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install pycryptodome pycryptodomex
! pip uninstall --yes pandas-datareader
! pip install git+https://github.com/raphi6/pandas-datareader.git@ea66d6b981554f9d0262038aef2106dda7138316

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.3 MB/s eta 0:00:00
Found existing installation: pandas-datareader 0.9.0
Uninstalling pandas-datareader-0.9.0:
  Successfully uninstalled pandas-datareader-0.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/raphi6/pandas-datareader.git (to revision ea66d6b981554f9d0262038aef2106dda7138316) to /tmp/pip-req-build-dkd4b4i3
  Running command git clone --filter=blob:none --quiet https://github.com/raphi6/pandas-datareader.git /tmp/pip-req-build-dkd4b4i3
  Running command git rev-parse -q --verify 'sha^ea66d6b981554f9d0262038aef2106dda7138316'
  Running command git fetch -q https://github.com/raphi6/pandas-datareader.git ea66d6b981554f9d0262038aef2106dda7138316
  Running comman

#Imports de bibliotecas

In [ ]:
import requests as req

import datetime as dt
import pandas as pd
import numpy as np

import statsmodels.api as sm

In [ ]:
import pandas_datareader as pdr
import investpy

import quandl
quandl.ApiConfig.api_key = "RMn6QX96UxspKjLtzpPF"

In [ ]:
from plotly import graph_objects as go
from plotly.tools import make_subplots

#Classes para busca de fatores do NEFIN

In [ ]:
class Portfolios():

    def __init__(self, *, verbose=False):
        self.size = self.set_portfolio('size', verbose=verbose)
        self.book_to_market = self.set_portfolio('book-to-market', verbose=verbose)
        self.momentum = self.set_portfolio('momentum', verbose=verbose)
        self.illiquidity = self.set_portfolio('illiquidity', verbose=verbose)


    def set_portfolio(self, name, portfolio_number=3, *, verbose=False):
        if verbose:
            print(f'Fetching {name} portfolios', end=' ---- ')
        url = f'https://nefin.com.br/data/Portfolios/{portfolio_number}_portfolios_sorted_by_{name}.xls'
        if verbose:
            print('OK')
        portfolio = pd.read_excel(url)
        portfolio.index = self._reformat_dates(portfolio)
        portfolio.drop(columns=['year', 'month', 'day'], inplace=True)
        portfolio.name = name
        return portfolio


    def _reformat_dates(self, portfolio):
        new_dates = [
        dt.date(portfolio['year'].iloc[i], portfolio['month'].iloc[i], portfolio['day'].iloc[i]) 
            for i in range(portfolio.shape[0])
        ]
        return new_dates

    def to_dict(self):
        return {
            'size': self.size,
            'book-to-market': self.book_to_market,
            'momentum': self.momentum,
            'illiquidity': self.illiquidity,
        }

    def get_portfolio_names(self):
        return list( self.to_dict().keys() )
    

In [ ]:
class Factors():
    def __init__(self, start=dt.date.today()-dt.timedelta(days=365*5), end=dt.date.today(), *, benchmark=None, auto_init=True, verbose=False):
        self.start = start
        self.end = end
        if auto_init:
            self.portfolios = Portfolios(verbose=verbose)
            self.MKT = self.set_market_factor(benchmark=benchmark, verbose=verbose)
            self.SMB = self.set_factor(self.portfolios.size, long_portfolio_column=0, short_portfolio_column=2, verbose=verbose)
            self.HML = self.set_factor(self.portfolios.book_to_market, long_portfolio_column=2, short_portfolio_column=0, verbose=verbose)
            self.WML = self.set_factor(self.portfolios.momentum, long_portfolio_column=2, short_portfolio_column=0, verbose=verbose)
            self.IML = self.set_factor(self.portfolios.illiquidity, long_portfolio_column=2, short_portfolio_column=0, verbose=verbose)


    def set_factor(self, portfolio, long_portfolio_column, short_portfolio_column, *, verbose=False):
        if verbose:
            print(f'Calculating {portfolio.name} factor', end=' ---- ')
        long_portfolio_name = portfolio.columns[long_portfolio_column]
        short_portfolio_name = portfolio.columns[short_portfolio_column]
        factor = portfolio[long_portfolio_name] - portfolio[short_portfolio_name]
        factor.name = portfolio.name
        if verbose:
            print('OK')

        return factor 

    def set_market_factor(self, start=None, end=None, *, benchmark=None, verbose=False):
        if start is None:
            start = self.start
        if end is None:
            end = self.end
        str_start = start.strftime('%d/%m/%Y')
        str_end = end.strftime('%d/%m/%Y')

        if verbose:
            print(f'Calculating Market Factor ---- Fetching benchmark:', end=' ')
        if benchmark is None:
            benchmark = pdr.get_data_yahoo('^BVSP', start, end)
            benchmark = benchmark['Adj Close'].pct_change()
        
        if verbose:
            print('OK ---- Fetching SELIC:', end=' ')
        selic = self.get_selic(start, end)
        if verbose:
            print('OK')
        MKT = benchmark - selic
        MKT.name = 'market'
        return MKT
    
    def get_selic(self, start=None, end=None):
        if start is None:
            start = self.start
        if end is None:
            end = self.end
        str_start = start.strftime('%d/%m/%Y')
        str_end = end.strftime('%d/%m/%Y')
        url = "http://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=csv&dataInicial="+ str_start +"&dataFinal="+str_end
        selic = pd.read_csv(url, sep=";", decimal=',')
        selic.index = [dt.datetime.strptime(date, '%d/%m/%Y')  for date in selic['data'] ]
        selic['valor'] = selic['valor']/100
        return selic['valor']


    def to_dataframe(self):
        df = pd.DataFrame({
            'MKT': self.MKT,
            'SMB': self.SMB,
            'HML': self.HML,
            'WML': self.WML,
            'IML': self.IML,
            })
        df.name = 'risk factors'
        return df

#Busca de Fatores

In [ ]:
# benchmark = pd.read_excel('drive/My Drive/ibov.xlsx', header=3, index_col=0, na_values='-')['Fechamento'].dropna()
# benchmark = benchmark.pct_change()
benchmark = None

In [ ]:
from_date = dt.datetime(2001,1,1)
to_date = dt.datetime.today()
factors = Factors(from_date, to_date, benchmark=benchmark, verbose=True)
factors_df = factors.to_dataframe()
# factors_df.dropna(inplace=True)

Fetching size portfolios ---- OK
Fetching book-to-market portfolios ---- OK
Fetching momentum portfolios ---- OK
Fetching illiquidity portfolios ---- OK
Calculating Market Factor ---- Fetching benchmark: OK ---- Fetching SELIC: OK
Calculating size factor ---- OK
Calculating book-to-market factor ---- OK
Calculating momentum factor ---- OK
Calculating illiquidity factor ---- OK


In [ ]:
factors_df.dropna()

,MKT,SMB,HML,WML,IML
2001-01-03,0.075528,0.005390,0.009390,-0.028644,0.004510
2001-01-04,0.003995,0.006690,-0.002327,-0.000946,-0.009227
2001-01-05,-0.016474,0.003523,-0.002397,0.005985,0.025124
2001-01-08,0.008680,0.007883,0.001948,-0.004099,-0.001175
2001-01-09,0.024415,0.005928,-0.000916,0.009564,-0.003978
...,...,...,...,...,...
2022-11-24,0.028457,0.008471,-0.009445,-0.011714,0.003420
2022-11-25,-0.026029,-0.001412,0.010969,0.004745,-0.003082
2022-11-28,-0.002297,-0.015643,0.005003,0.026090,-0.007774
2022-11-29,0.019054,0.006213,0.008128,0.000526,0.006622


In [ ]:
factors_df.to_excel('drive/My Drive/fatores_de_risco.xlsx')

#Avaliação de Portfólio

In [ ]:
def factor_evaluation(portfolio:pd.Series, factors:pd.DataFrame):
    df = factors.copy()
    df['portfolio'] = portfolio
    df = df.dropna()

    X = sm.add_constant(df[factors.columns])
    X.rename(columns={'const':'Alpha'}, inplace=True)
    y = df['portfolio']

    model = sm.OLS(y, X).fit()
    return model
    

In [ ]:
portfolio = pd.read_excel('drive/My Drive/CSHG EC CAPITAL FIM CP IE.xlsx', header=3, index_col=0, na_values='-')
portfolio_returns = portfolio['Cota'].pct_change()


In [ ]:
model = factor_evaluation(portfolio_returns, factors_df)
print(model.summary2())

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.355      
Dependent Variable: portfolio        AIC:                -10457.8642
Date:               2023-01-09 13:09 BIC:                -10424.0698
No. Observations:   2064             Log-Likelihood:     5234.9     
Df Model:           5                F-statistic:        228.3      
Df Residuals:       2058             Prob (F-statistic): 3.51e-194  
R-squared:          0.357            Scale:              0.00036795 
-----------------------------------------------------------------------
          Coef.     Std.Err.       t       P>|t|      [0.025     0.975]
-----------------------------------------------------------------------
Alpha     0.0021      0.0004     5.0285    0.0000     0.0013     0.0030
MKT       0.8194      0.0296    27.7178    0.0000     0.7614     0.8774
SMB       0.5476      0.1027     5.3342    0.0000     0.3463     0.7489
HML      -0.3575      0.0539    -6.

In [ ]:
start = portfolio_returns.index[0]
end = factors_df.index[-1]

In [ ]:
factors_slice = factors_df.iloc[factors_df.index.get_loc(start, method='pad'):]

In [ ]:
factors_daily_mean = (factors_slice+1).cumprod().iloc[-1]**(1/factors_slice.shape[0])-1
factors_annual_mean = (factors_daily_mean+1)**252 -1
alpha = (model.params['Alpha']+1)**252 -1
factors_annual_mean

MKT   -0.008015
SMB         NaN
HML         NaN
WML         NaN
IML         NaN
Name: 2023-01-06 00:00:00, dtype: float64

In [ ]:
beta_values = dict()

for coef, coef_value in model.params.items():
    if coef == 'Alpha':
        value = alpha
    else:
        value = coef_value*factors_annual_mean[coef]
    beta_values[coef] = value
beta_values = pd.Series(beta_values)*100
beta_values.sort_values(ascending=False, inplace=True)

In [ ]:
beta_values['Retorno Esperado'] = beta_values.sum()
beta_values

Alpha               71.056145
MKT                 -0.656801
SMB                       NaN
HML                       NaN
WML                       NaN
IML                       NaN
Retorno Esperado    70.399344
dtype: float64

In [ ]:
fig = go.Figure()

base = 0

for coef, coef_value in beta_values.items():
    if coef == 'Retorno Esperado':
        base=0
    bar = go.Bar(
        x=[coef],
        y=[coef_value],
        name=coef,
        base=base
    )
    base += coef_value
    fig.add_trace(bar)

fig.update_layout(
    template='plotly_white',
    title_text=f"Composição do Retorno Anual Esperado <br><sup>Dados entre {start.strftime('%d/%m/%Y')} e {end.strftime('%d/%m/%Y')}</sup> <br><sup>Fontes: TC Economatica | NEFIN FEA-USP</sup>",
    height=400, width=800,
    # xaxis_title="Tempo",
    yaxis_title='Retorno (%)',
    # legend_title='Tradings',
)

fig.show()

In [ ]:
color = {
    'MKT': 'blue',
    'SMB': 'brown',
    'HML': 'gray',
    'WML': 'orange',
    'IML': 'purple',
    'Alpha': 'green',

}

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=('Coeficientes', 'P-Valores'), shared_xaxes=True)

for coef, coef_value in model.pvalues.items():
    bar = go.Bar(
        x = [coef],
        y = [coef_value],
        marker_color=color[coef],
        showlegend=False,
    )
    fig.add_trace(bar, row=2, col=1)

signifiant_limit = go.Scatter(
    x = model.tvalues.index,
    y = [0.05 for _ in model.tvalues],
    marker_color='red',
    name = 'Limite de<br>significância (0.05)',
    mode='lines'
)
fig.add_trace(signifiant_limit, row=2, col=1)

for coef, coef_value in model.params.sort_values(ascending=True).items():
    if coef == 'Alpha':
        coef_value = alpha
    bar = go.Bar(
        x = [coef],
        y = [coef_value],
        marker_color=color[coef],
        showlegend=False,
    )
    fig.add_trace(bar, row=1, col=1)


fig.update_layout(
    template='plotly_white',
    title_text=f"Coeficientes do Modelo Multifatorial<br><sup>Fontes: TC Economatica | NEFIN FEA-USP</sup>",
    height=450, width=800,
    # xaxis_title="Tempo",
    # yaxis_title='Retorno (%)',
    # legend_title='Tradings',
)


fig.show()

/usr/local/lib/python3.8/dist-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#Alphas de Fundos

In [ ]:
def get_portfolios_alphas(portfolios:pd.DataFrame, factors:pd.DataFrame, use_factors=None):
    alphas = pd.Series(dtype=float)
    if use_factors is not None:
        factors = factors[use_factors]
    for port_name, port in portfolios.items():
        model = factor_evaluation(port, factors)
        alpha = model.params['Alpha']
        alphas.loc[port_name] = alpha
    
    return alphas


In [ ]:
fia = pd.read_excel('drive/My Drive/FIA 2022.xlsx', header=3, index_col=0, na_values='-')
fia.columns = [ col.split('\n')[-1] for col in fia.columns]
fia.rename(columns={'ajust p/ prov': '558060'}, inplace=True)
fia.rename(columns={col: int(col) for col in fia.columns}, inplace=True)
fia

,558060,213691,499943,485675,505412,507113,428582,522211,522627,522635,...,603635,561452,558834,530727,531510,367222,436100,316873,171379,450278
Data,,,,,,,,,,,,,,,,,,,,,
2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,-0.135340,-1.618674,-1.421661,-1.417179,-1.972615,-1.903569,1.550921,-3.479683,-1.097645,-1.231168,...,0.557974,0.037951,1.593273,0.020963,1.633445,1.672281,-2.003541,-2.503301,-1.985157,-0.914724
2022-01-04,-0.790740,-2.208887,-0.776761,-0.771906,-1.267577,-1.226752,-0.466342,-1.372231,0.174987,-0.935669,...,0.021238,-0.917379,-0.548745,-0.932498,-0.562581,-0.412487,-1.091579,-1.266471,-1.100127,-0.370826
2022-01-05,-2.553243,-4.754327,-2.375315,-2.381655,-4.121975,-3.988345,-0.754209,-3.849242,-1.711230,-3.615083,...,-1.431171,-2.108572,-2.255940,-2.165596,-2.319760,-2.604610,-2.449968,-2.085518,-2.599320,-2.414871
2022-01-06,-0.709364,-1.624070,0.355291,0.355959,0.663711,0.642660,-0.412099,-0.833890,-0.614884,-0.102597,...,-0.409246,-0.812850,-0.667008,-0.838568,-0.697118,-0.420500,0.092803,-0.036025,0.103968,0.518563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
alphas_multfac = get_portfolios_alphas(fia, factors_df)
alphas_jensen = get_portfolios_alphas(fia, factors_df, use_factors=['MKT'])

alphas_jensen.sort_values(ascending=False, inplace=True)
alphas_multfac.sort_values(ascending=False, inplace=True)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
len(alphas_jensen)

428

In [ ]:
codigo = pd.read_excel('drive/My Drive/fia - codigo.xlsx', header=3, index_col=0, na_values='-')
codigo.columns = [col.split('\n')[-1] for col in codigo.columns]
codigo = codigo[pd.Series([cod.isdigit() for cod in codigo['Anbima']], index=codigo.index)]

In [ ]:
codigo.index = [int(i) for i in codigo['Anbima'].values]
codigo = codigo['Nome']
codigo

511692                              051 Acoes FIA
653500          051 Agro Fiagro Imobiliario-Unica
540293        051 Allocation FICFI Mult Cred Priv
563404                     051 Alocacao Acoes FIA
542407    051 Alocacao Moder FICFI Mult Cred Priv
                           ...                   
609145                Zurich FI Mult Cred Priv Ie
478709                    Zurich Schroder FI Mult
490938      Zurich Valora Prev FICFI RF Cred Priv
619256          Zurique Deb Incent FI em Infra RF
439959      Zw Investimentos FICFI Mult Cred Priv
Name: Nome, Length: 31150, dtype: object

In [ ]:
results = pd.DataFrame(index=codigo.index)

results['Nome'] = codigo
results['Alpha Multi-Fatorial (até 31/10/2022)'] = alphas_multfac
results['Alpha de Jensen (até 14/12/2022)'] = alphas_jensen

results = results[results['Alpha Multi-Fatorial (até 31/10/2022)'].notna()]

In [ ]:
results.sort_values('Alpha Multi-Fatorial (até 31/10/2022)', ascending=False, inplace=True)

In [ ]:
results.to_excel('drive/My Drive/Alphas.xlsx')